In [2]:
import os
import time
import openai
import re
import requests
import warnings
import pandas as pd
from bs4 import BeautifulSoup

# Set your OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = #YOUR API KEY
openai.api_key = #YOUR API KEY


The following function will go into a link and scrape all <body> elements.
This is a potential place for improvement, since we could be selectively just looking at the important bits.
If we skip header, footer, nav menus etc, we would save GPT credits, time and money.

In [3]:

def extract_text_from_website(url):
    response = requests.get(url, verify=False)
    html = response.text
    soup = BeautifulSoup(html, 'lxml')  # Use 'lxml' or 'html5lib' as the parser

    # Find all "body" elements and extract their text
    body_elements = soup.find_all('body')
    text = "\n".join([body.get_text() for body in body_elements])

    # Remove repeated newline characters (more than two in a row)
    text = re.sub(r'\n{2,}', '\n\n', text)

    return text

This function does the API call. I found that using gpt3.5turbo would give poor results.
Promt engineering should definitely be improved before next run, but has been unchanged since last run.
"Type of Grant" category should include "disabled facilities" as this is super common, contains boiler repairs and often confuses the AI

In [4]:
def get_gpt4_response(url, LA_name):
    result = extract_text_from_website(url)
    prompt = "The text you will look at is a parsed body of text from a website of a Local Council, which may or may not contain important information about grants available to homeowners. Your task is to understand available grants and log them into an important database. Based on this text parsed from the HTML file, give me a table with the following fields: Name of grant, Type of Grant (Whether it is Home Renewables, Home Energy Efficiency or Other), short grant description, URL of the grant, the amount of funding if available, list of measures that can be installed under this grant, start of funding date if available, end of funding date if available, conditions for eligibility and also any other notes. The columns will have the following names: [Name of Grant, Type of Grant, Grant Description, Grant URL, Amount of Funding, Measures, Start Date, End Date, Conditions for Eligibility, Other Notes]. If you struggle to fill any field of the table write NA. The values in Measure column must belong strictly to one of these categories: [Draft proof your external doors, Draft proof your windows, Install a biomass boiler (wood pellets), Install a gas combi boiler, Install a gas condensing boiler, Install a ground source heat pump, Install a log stove, Install a LPG combi boiler, Install a LPG condensing boiler, Install a new hot water tank, Install a solar hot water system, Install additional thermostatic controls, warm air systems, Install an air source heat pump, Install an oil combi boiler, Install an oil combi boiler (plus oil storage tank), Install an oil condensing boiler, Install an oil condensing boiler (plus oil storage tank), Install A-rated glazing (uPVC), Install cavity wall insulation, Install hot water tank insulation, Install hot water tank insulation and new controls, Install improved hot water controls, Install insulation for flat roofing, Install loft insulation, Install modern storage heaters, Install new insulated uPVC external doors, Install new radiators and distribution system, Install party wall insulation, Install room-in roof insulation, Install secondary glazing, Install solar PV panels, Install solid floor insulation, Install solid wall insulation, Install storage heater Celect type controls, Install suspended wooden floor insulation, Install thermostatic radiator valves, Install underfloor heating, Time and temperature zone control, Top-up your loft insulation, All heating measures, All insulation, All renewables, Non renewables]. The text to search is here: "
    system_msg = 'You are a helpful assistant mapping grant schemes available across the UK. You write grants from these schemes into an important database'
    user_msg = prompt + result
    response = openai.ChatCompletion.create(model="gpt-4",
                                            messages=[{"role": "system", "content": system_msg},
                                                      {"role": "user", "content": user_msg}],
                                            temperature=1)
    return(response)

This function parses the API response and gets the table we require, saving it into a csv line by line.

In [5]:
def extract_and_append_to_csv(api_output, csv_filename, link, name):
    # Check if the CSV file already exists, and create it with headers if it doesn't
    try:
        existing_data = pd.read_csv(csv_filename)
    except FileNotFoundError:
        existing_data = pd.DataFrame(columns=["Name of Grant", "Grant Type", "Grant Description", "Grant URL", "Amount of Funding", "Measures", "Start Date", "End Date", "Conditions for Eligibility", "Other Notes"])
        existing_data.to_csv(csv_filename, index=False)

    # Extract the content
    content = api_output["choices"][0]["message"]["content"]

    # Split the content into rows
    rows = content.split('\n')

    # Initialize a list to hold the data
    table_data = []
    header = None

    for row in rows:
        row = row.strip()
        if row.startswith('|'):
            # Extract columns using the pipe symbol as the delimiter
            columns = [column.strip() for column in re.split(r'\s*\|\s*', row) if column.strip()]
            if not header:
                header = columns
            else:
                if len(columns) == len(header):
                    table_data.append(columns)

    # Create a Pandas DataFrame with column names
    df = pd.DataFrame(table_data, columns=header)
    df['Link'] = link
    df['Local Authority'] = name
    df['Date Updated'] = pd.Timestamp("today").strftime("%d/%m/%Y")
    filtered_df = df[~df.applymap(lambda x: isinstance(x, str) and '---' in x).any(axis=1)]

    # Append the data to the existing CSV file
    filtered_df.to_csv(csv_filename, mode='a', header=False, index=False)

In [6]:
websites = pd.read_csv("search_results.csv")
test_urls = websites["URL"]
test_councils = websites["Council Name"]

In [8]:
print(len(test_councils))

1438


This part of the code takes quite a while to run so likely, you will want to split things up into multiple files and then merge them manually at the end.
Some errors still happen, mostly when the google search leads to a PDF. These are all noted in the error file and could be explored manually separately if time permits.

In [11]:
warnings.filterwarnings('ignore')
csv_filename = "FullRun_011123_2.csv"
issues = {'council':[],'error':[],'url':[]}
for i in range(1400, 1438):
    try:
        response = get_gpt4_response(test_urls[i], test_councils[i])
        extract_and_append_to_csv(response, csv_filename, test_urls[i], test_councils[i])
    except Exception as e:
        print(f"Error processing url {test_urls[i]}: {e}")
        try:
            time.sleep(60)
            response = get_gpt4_response(test_urls[i], test_councils[i])
            extract_and_append_to_csv(response, csv_filename, test_urls[i], test_councils[i])
        except Exception as e:
            print(f"Error re-processing url{test_urls[i]}: {e}")
            issues['council'].append(test_councils[i])
            issues['error'].append(f"Error re-processing url{test_urls[i]}: {e}")
            issues['url'].append(test_urls[i])
            extract_and_append_to_csv(response, csv_filename, test_urls[i], test_councils[i])

    print(i)

issues_df = pd.DataFrame(issues)
issues_df.to_csv("error_log_011123_2.csv", mode='a', index=False)





1400
1401
1402
1403
1404
1405
1406
Error processing url https://www.tmbc.gov.uk/downloads/file/3011/housing-assistance-policy: Rate limit reached for gpt-4 in organization org-yIC1FzVuNadajuXe1tIE6di4 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Visit https://platform.openai.com/account/rate-limits to learn more.
Error re-processing urlhttps://www.tmbc.gov.uk/downloads/file/3011/housing-assistance-policy: Rate limit reached for gpt-4 in organization org-yIC1FzVuNadajuXe1tIE6di4 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Visit https://platform.openai.com/account/rate-limits to learn more.
1407
1408
1409
1410
1411
1412
1413
1414
Error processing url https://www.warrington.gov.uk/sites/default/files/2019-08/low_cost_loans_info_sheet_august_13.pdf: Rate limit reached for gpt-4 in organization org-yIC1FzVuNadajuXe1tIE6di4 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Visit https://platform.openai.com/account/rate-limits to learn more